# Librerías

In [ ]:
import pandas as pd
import pickle

In [ ]:
!pip install tinyec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for tinyec: filename=tinyec-0.4.0-py3-none-any.whl size=20877 sha256=d1e48ce7ea75951bf82ecfbe10d24889c8cf1b35fbd5829db978c482b37768b7
  Stored in directory: /root/.cache/pip/wheels/02/37/a5/aa011cfa66451de6aa2dbccaa3e7862e8290f0946653753265
Successfully built tinyec


In [ ]:
!pip install pycryptodome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Base de datos

In [ ]:
dataset = pd.read_csv('/content/drive/Shareddrives/Cripto 3/reto/Prosumer_ABC.csv', header= 0, sep = ";" )

In [ ]:
dataset.head(3)

,ID,Consumo (0) / Producción (1),Dia,Mes,Año,1,2,3,4,5,...,87,88,89,90,91,92,93,94,95,96
0,ABC,0,2,11,2013,58.00,75.0,65.0,0.08,67.0,...,338.0,369.0,318.0,322.0,345.0,298.0,316.0,299.0,341.00,304.0
1,ABC,0,3,11,2013,0.16,0.1,68.0,69.00,75.0,...,67.0,74.0,61.0,84.0,74.0,67.0,77.0,91.0,0.07,61.0
2,ABC,0,4,11,2013,0.08,66.0,77.0,64.00,84.0,...,114.0,106.0,128.0,511.0,74.0,84.0,59.0,69.0,69.00,71.0


# Hybrid ECDH with AES

In [ ]:
# Descarga de librerías para la generación de llaves
from tinyec import registry
import secrets

Código obtenido de https://github.com/nakov/Practical-Cryptography-for-Developers-Book/blob/master/asymmetric-key-ciphers/ecc-encryption-decryption.md

Para su comprensión y futura programación propia

## Encriptación y desencriptación

In [ ]:
# ---------- ECC-Based Hybrid Encryption / Decryption -----------------------

from tinyec import registry
from Crypto.Cipher import AES
import hashlib, secrets, binascii

# Encriptación del mensaje usando AES; requiere la clave privada y retorna el texto cifrado, vector de inicialización y código de autenticación
def encrypt_AES_GCM(msg, secretKey):
    aesCipher = AES.new(secretKey, AES.MODE_GCM)
    ciphertext, authTag = aesCipher.encrypt_and_digest(msg)
    return (ciphertext, aesCipher.nonce, authTag)

# Desencriptación del mensaje
def decrypt_AES_GCM(ciphertext, nonce, authTag, secretKey):
    aesCipher = AES.new(secretKey, AES.MODE_GCM, nonce)
    plaintext = aesCipher.decrypt_and_verify(ciphertext, authTag)
    return plaintext

# Hashing
def ecc_point_to_256_bit_key(point):
    sha = hashlib.sha256(int.to_bytes(point.x, 32, 'big'))
    sha.update(int.to_bytes(point.y, 32, 'big'))
    return sha.digest()

curve = registry.get_curve('brainpoolP256r1')

# Cifrado con curvas elípticas y AES; retorna lo mismo que encrypt_AES_GCM más la clave pública
def encrypt_ECC(msg, pubKey):
    ciphertextPrivKey = secrets.randbelow(curve.field.n) # b
    sharedECCKey = ciphertextPrivKey * pubKey # a * g * b
    secretKey = ecc_point_to_256_bit_key(sharedECCKey) 
    ciphertext, nonce, authTag = encrypt_AES_GCM(msg, secretKey)
    ciphertextPubKey = ciphertextPrivKey * curve.g # b * g
    return (ciphertext, nonce, authTag, ciphertextPubKey)
    

def encrypt_ECC_shared(msg, pubKey):
    ciphertextPrivKey = secrets.randbelow(curve.field.n) # b
    sharedECCKey = ciphertextPrivKey * pubKey # a * g * b
    secretKey = ecc_point_to_256_bit_key(sharedECCKey) 
    ciphertext, nonce, authTag = encrypt_AES_GCM(msg, secretKey)
    ciphertextPubKey = ciphertextPrivKey * curve.g # b * g
    return (sharedECCKey)  


# Desencriptación
def decrypt_ECC(encryptedMsg, privKey):
    (ciphertext, nonce, authTag, ciphertextPubKey) = encryptedMsg
    sharedECCKey = privKey * ciphertextPubKey # a * b * g
    secretKey = ecc_point_to_256_bit_key(sharedECCKey)
    plaintext = decrypt_AES_GCM(ciphertext, nonce, authTag, secretKey)
    return plaintext

In [ ]:
encrypt_ECC_shared(msg, pubKey)

(30628338167825922613858556929443490703911812388008836112799285310187983676926, 21125571972223611259174231022480224784089453711563656535928925101045408475257) on "brainpoolP256r1" => y^2 = x^3 + 56698187605326110043627228396178346077120614539475214109386828188763884139993x + 17577232497321838841075697789794520262950426058923084567046852300633325438902 (mod 76884956397045344220809746629001649093037950200943055203735601445031516197751)

### Encriptación

In [ ]:
privKey = secrets.randbelow(curve.field.n)
pubKey = privKey * curve.g

# Se convierte la base de datos a bytes para su manejo
msg = pickle.dumps(dataset, protocol=4)

encryptedMsg = encrypt_ECC(msg, pubKey)
encryptedMsgObj = {
    'ciphertext': binascii.hexlify(encryptedMsg[0]),
    'nonce': binascii.hexlify(encryptedMsg[1]),
    'authTag': binascii.hexlify(encryptedMsg[2]),
    'ciphertextPubKey': hex(encryptedMsg[3].x) + hex(encryptedMsg[3].y % 2)[2:]
}

In [ ]:
print("encrypted msg:", encryptedMsgObj)

encrypted msg: {'ciphertext': b'...', 'nonce': b'cf40b67f81f7a10eea931594678d7abf', 'authTag': b'4802f65a52bb4fd9b6162c336e7415e0', 'ciphertextPubKey': '0x3381eaec37dbf99f5236d0b29acb7c42006a8f068591fd1587f75f21610655451'}


### Desencriptación

En bytes

In [ ]:
decryptedMsg = decrypt_ECC(encryptedMsg, privKey)
print("decrypted msg:", decryptedMsg)

decrypted msg: b'\x80\x04\x95\x8db\x00\x00\x00\x00\x00\x00\x8c\x11pandas.core.frame\x94\x8c\tDataFrame\x94\x93\x94)\x81\x94}\x94(\x8c\x04_mgr\x94\x8c\x1epandas.core.internals.managers\x94\x8c\x0cBlockManager\x94\x93\x94\x8c\x16pandas._libs.internals\x94\x8c\x0f_unpickle_block\x94\x93\x94\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01K\x01M\xd8\x02\x86\x94h\x0f\x8c\x05dtype\x94\x93\x94\x8c\x02O8\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01|\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK?t\x94b\x89]\x94(\x8c\x03ABC\x94h\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1fh\x1f

Base de datos

In [ ]:
pickle.loads(decryptedMsg).head()

,ID,Consumo (0) / Producción (1),Dia,Mes,Año,1,2,3,4,5,...,87,88,89,90,91,92,93,94,95,96
0,ABC,0,2,11,2013,58.00,75.0,65.0,0.08,67.0,...,338.0,369.0,318.00,322.0,345.0,298.00,316.00,299.0,341.00,304.0
1,ABC,0,3,11,2013,0.16,0.1,68.0,69.00,75.0,...,67.0,74.0,61.00,84.0,74.0,67.00,77.00,91.0,0.07,61.0
2,ABC,0,4,11,2013,0.08,66.0,77.0,64.00,84.0,...,114.0,106.0,128.00,511.0,74.0,84.00,59.00,69.0,69.00,71.0
3,ABC,0,5,11,2013,68.00,58.0,95.0,81.00,74.0,...,167.0,202.0,0.53,226.0,256.0,315.00,421.00,66.0,0.09,64.0
4,ABC,0,6,11,2013,77.00,61.0,97.0,65.00,78.0,...,117.0,345.0,351.00,69.0,57.0,0.07,0.07,72.0,56.00,78.0


# Elliptic curves with ElGamal

## Generación de llaves

In [ ]:
# ---------------------- ECC-Based Secret Key Derivation ----------------------

# Generar la curva elíptica tipo brainpoolP256r1
curve = registry.get_curve('brainpoolP256r1')
"""
P     Field type = Prime field
256   Key size = 256
r     Curve type = Regular curve
1     Cofactor = 1
"""

# Transformación a hexadecimal para no ocupar tanto espacio en la memoria
def compress_point(point):
    return hex(point.x) + hex(point.y % 2)[2:]

# Cálculo de las llaves para la encriptación (esquema ECDH, llave pública)
def ecc_calc_encryption_keys(pubKey):
    # Llave privada aleatoria
    ciphertextPrivKey = secrets.randbelow(curve.field.n) # b
  
    # Creación de la llave publica a partir del producto entre el generador de la curva y la llave privada
    ciphertextPubKey = ciphertextPrivKey * curve.g  # b * g 
  
    # Calcula el esquema Elliptic Curve Diffie–Hellman con ecuación (a * G) * b
    sharedECCKey = pubKey * ciphertextPrivKey # (a * g) * b
    return (sharedECCKey, ciphertextPubKey)

# Cálculo del esquema ECDH para la desencriptación
def ecc_calc_decryption_key(privKey, ciphertextPubKey):
  # Con la ecuación (b * g) * a
    sharedECCKey = ciphertextPubKey * privKey
    return sharedECCKey

Llaves

In [ ]:
# Retorna un entero random en el rango de [0,curve.field.n] (curve.field.n = orden de la curva)
privKey = secrets.randbelow(curve.field.n) # a
# pubKey obtenida de la privKey y el Generador (g)
pubKey = privKey * curve.g # a * g
print("private key:", hex(privKey))
print("public key:", compress_point(pubKey))

(encryptKey, ciphertextPubKey) = ecc_calc_encryption_keys(pubKey)
print("ciphertext pubKey:", compress_point(ciphertextPubKey))
print("encryption key:", compress_point(encryptKey))

decryptKey = ecc_calc_decryption_key(privKey, ciphertextPubKey)
print("decryption key:", compress_point(decryptKey))

private key: 0x25f222831c3040e7f3f5d12c96438f48a81b9369c94049890095846d9541ce2e
public key: 0x88b0bc953de4100ae9e837a9fa7d039da30af976f0019b919563fa506c6540470
ciphertext pubKey: 0x10966215b3c7d888fd9f2231d22ca98a74498b0a71ffe7ee053de671181c328b0
encryption key: 0x9a5516ba06ccfd279ae81e11aadc4053a7c57f14eaa80aef4365fd844df37ec11
decryption key: 0x9a5516ba06ccfd279ae81e11aadc4053a7c57f14eaa80aef4365fd844df37ec11


# Verificación

In [ ]:
import time


Paper donde lo mencionan:
http://article.nadiapub.com/IJSIA/vol11_no3/2.pdf 

## Correlación entre texto cifrado y original

https://www.geeksforgeeks.org/avalanche-effect-in-cryptography/ 

## Tiempo de encriptación

In [ ]:
def time_encrypt(dataset):
  inicio = time.time()
  privKey = secrets.randbelow(curve.field.n)
  pubKey = privKey * curve.g

  # Se convierte la base de datos a bytes para su manejo
  msg = pickle.dumps(dataset, protocol=4)

  encryptedMsg = encrypt_ECC(msg, pubKey)
  encryptedMsgObj = {
      'ciphertext': binascii.hexlify(encryptedMsg[0]),
      'nonce': binascii.hexlify(encryptedMsg[1]),
      'authTag': binascii.hexlify(encryptedMsg[2]),
      'ciphertextPubKey': hex(encryptedMsg[3].x) + hex(encryptedMsg[3].y % 2)[2:]
  }
  fin = time.time()
  return fin-inicio

## Tiempo de desencriptación

In [ ]:
def time_decrypt(encryptedMsg, privKey);
  inicio = time.time()
  decryptedMsg = decrypt_ECC(encryptedMsg, privKey)
  pickle.loads(decryptedMsg)
  fin = time.time()
  return fin-inicio

SyntaxError: ignored

## Almacenamiento
Comportamiento del tamaño del archivo después de la encriptación 

# Private and public key files

In [ ]:
from cryptography.hazmat.backends import default_backend  
from cryptography.hazmat.primitives import serialization  
from cryptography.hazmat.primitives.asymmetric import rsa  
  
  
# save file helper  
def save_file(filename, content):  
   f = open(filename, "wb")  
   f.write(content) 
   f.close()  
  
  
# generate private key & write to disk  
private_key = rsa.generate_private_key(  
    public_exponent=65537,  
    key_size=4096,  
    backend=default_backend()  
)  
pem = private_key.private_bytes(  
    encoding=serialization.Encoding.PEM,  
    format=serialization.PrivateFormat.PKCS8,  
    encryption_algorithm=serialization.NoEncryption()  
)  
save_file("private.pem", pem)  
  
# generate public key  
public_key = private_key.public_key()  
pem = public_key.public_bytes(  
    encoding=serialization.Encoding.PEM,  
    format=serialization.PublicFormat.SubjectPublicKeyInfo  
)  
save_file("public.pem", pem)  

In [ ]:
f = open('priv.pem', "wb")  
f.write(priv_b) 
f.close()  

In [ ]:
curve = registry.get_curve('brainpoolP256r1')
privKey = secrets.randbelow(curve.field.n)
pubKey = privKey * curve.g

save_file("priv.pem",priv_b)

In [ ]:
priv_b = pickle.dumps(privKey, protocol=4)

In [ ]:
pem

In [ ]:
private_key

In [ ]:
priv_b